In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt 
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from xgboost import XGBClassifier
import mplleaflet

In [2]:
trainDataset=pd.read_csv(r'D:/Academics/SEM 7/Final year project/Dataset/raw.csv')
trainDataset.head()

,cmplnt_num,addr_pct_cd,cmplnt_fr_dt,cmplnt_fr_tm,crm_atpt_cptd_cd,juris_desc,ky_cd,law_cat_cd,loc_of_occur_desc,ofns_desc,...,pd_desc,prem_typ_desc,susp_age_group,susp_race,susp_sex,housing_psa,station_name,transit_district,hadevelopt,parks_nm
0,758521794,67,2021-03-07T00:00:00.000,01:50:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,696896951,60,2021-03-26T00:00:00.000,18:40:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,972479923,75,2021-03-27T00:00:00.000,15:55:00,COMPLETED,N.Y. POLICE DEPT,113,FELONY,FRONT OF,FORGERY,...,"FORGERY,ETC.,UNCLASSIFIED-FELO",DEPARTMENT STORE,25-44,WHITE HISPANIC,M,NaN,NaN,NaN,NaN,NaN
3,109344500,77,2021-03-28T00:00:00.000,11:10:00,COMPLETED,N.Y. POLICE DEPT,109,FELONY,FRONT OF,GRAND LARCENY,...,"LARCENY,GRAND OF VEHICULAR/MOTORCYCLE ACCESSORIES",STREET,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,673945415,23,2021-03-29T00:00:00.000,23:13:00,COMPLETED,N.Y. POLICE DEPT,101,FELONY,OUTSIDE,MURDER & NON-NEGL. MANSLAUGHTER,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
trainDataset=trainDataset[['cmplnt_fr_dt','cmplnt_fr_tm','law_cat_cd','ofns_desc','latitude','longitude','boro_nm']]
trainDataset.head()

,cmplnt_fr_dt,cmplnt_fr_tm,law_cat_cd,ofns_desc,latitude,longitude,boro_nm
0,2021-03-07T00:00:00.000,01:50:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.654223,-73.911563,NaN
1,2021-03-26T00:00:00.000,18:40:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.575443,-73.987795,NaN
2,2021-03-27T00:00:00.000,15:55:00,FELONY,FORGERY,40.678643,-73.867543,BROOKLYN
3,2021-03-28T00:00:00.000,11:10:00,FELONY,GRAND LARCENY,40.668584,-73.926980,BROOKLYN
4,2021-03-29T00:00:00.000,23:13:00,FELONY,MURDER & NON-NEGL. MANSLAUGHTER,40.793026,-73.942156,NaN


In [4]:
list_law_cat_cd=list(trainDataset['law_cat_cd'].unique())
global dict_law_cat_cd
dict_law_cat_cd = {}
for i in list_law_cat_cd:
    dict_law_cat_cd[i] = list_law_cat_cd.index(i)
    
list_ofns_desc=list(trainDataset['ofns_desc'].unique())
global dict_ofns_desc
dict_ofns_desc = {}
for i in list_ofns_desc:
    dict_ofns_desc[i] = list_ofns_desc.index(i)

trainDataset=trainDataset.fillna("OTHERS")
global dict_boro_nm
dict_boro_nm = {'OTHERS':0,'BROOKLYN':1,'QUEENS':2,'BRONX':3,'MANHATTAN':4,'STATEN ISLAND':5}

In [5]:
def preprocessTrain(p_df):
    p_df['datetime']=pd.to_datetime(p_df['cmplnt_fr_dt'] + ' ' + p_df['cmplnt_fr_tm'],errors='coerce')
    p_df=p_df.drop(columns=['cmplnt_fr_dt','cmplnt_fr_tm'])
    p_df=p_df.sort_values(by='datetime',ascending=True) 

    p_df['year']=pd.DatetimeIndex(p_df['datetime']).year
    p_df['month'] = pd.DatetimeIndex(p_df['datetime']).month
    p_df['weekday'] = p_df['datetime'].dt.dayofweek
    p_df['Hour'] = p_df.datetime.dt.hour
    p_df['Minute'] = p_df.datetime.dt.minute
    p_df=p_df.drop(columns=['datetime'])

    p_df['law_cat_cd']=p_df['law_cat_cd'].map(dict_law_cat_cd)

    p_df['ofns_desc']=p_df['ofns_desc'].map(dict_ofns_desc)
#     p_df['ofns_desc']=p_df['ofns_desc'].astype(int)

    p_df=p_df[p_df['year'].notna()]
    
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm)

    return p_df

In [6]:
dataTrain = preprocessTrain(trainDataset) 

print(dataTrain.describe())
dataTrain.head()

          law_cat_cd      ofns_desc       latitude      longitude  \
count  204643.000000  204636.000000  204643.000000  204643.000000   
mean        0.848912       9.389951      40.736213     -73.923832   
std         0.682593       6.235332       0.083193       0.077488   
min         0.000000       0.000000      40.499625     -74.253767   
25%         0.000000       4.000000      40.674009     -73.970767   
50%         1.000000       7.000000      40.733990     -73.926714   
75%         1.000000      13.000000      40.811556     -73.880916   
max         2.000000      59.000000      40.912723     -73.700720   

             boro_nm           year          month        weekday  \
count  204643.000000  204643.000000  204643.000000  204643.000000   
mean        2.537355    2020.924517       3.782641       2.967084   
std         1.256488       0.923219       2.091117       1.979039   
min         0.000000    1921.000000       1.000000       0.000000   
25%         1.000000    2021.0000

,law_cat_cd,ofns_desc,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute
92521,1,4.0,40.748923,-73.892230,2,1921.0,1.0,6.0,8.0,50.0
95915,1,11.0,40.652309,-73.956290,1,1921.0,1.0,3.0,22.0,0.0
44754,1,13.0,40.840984,-73.925974,3,1921.0,2.0,0.0,23.0,47.0
23354,0,2.0,40.758800,-73.890222,2,1921.0,3.0,2.0,4.0,4.0
190767,1,10.0,40.511804,-74.250035,5,1955.0,1.0,5.0,0.0,1.0


In [7]:
testDataset=pd.read_csv(r'D:/Academics/SEM 7/Final year project/Dataset/testDataset.csv')
testDataset.head()

,Unnamed: 0,cmplnt_fr_dt,cmplnt_fr_tm,latitude,longitude,boro_nm
0,0,01-08-21,00:00.0,40.861921,-73.891913,BRONX
1,1,01-08-21,12:12.8,40.760243,-73.928453,QUEENS
2,2,01-08-21,24:25.5,40.855710,-73.914939,BRONX
3,3,01-08-21,36:38.3,40.736739,-73.856241,QUEENS
4,4,01-08-21,48:51.1,40.846246,-73.856467,BRONX


In [8]:
def preprocessTest(p_df):
    p_df['datetime']=pd.to_datetime(p_df['cmplnt_fr_dt'] + ' ' + p_df['cmplnt_fr_tm'],errors='coerce')
    p_df=p_df.drop(columns=['cmplnt_fr_dt','cmplnt_fr_tm'])
    p_df=p_df.sort_values(by='datetime',ascending=True) 

    p_df['year']=pd.DatetimeIndex(p_df['datetime']).year
    p_df['month'] = pd.DatetimeIndex(p_df['datetime']).month
    p_df['weekday'] = p_df['datetime'].dt.dayofweek
    p_df['Hour'] = p_df.datetime.dt.hour
    p_df['Minute'] = p_df.datetime.dt.minute
    p_df=p_df.drop(columns=['datetime'])
    
    del p_df['Unnamed: 0']

    p_df=p_df[p_df['year'].notna()]
    
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm)

    return p_df

In [9]:
dataTest = preprocessTest(testDataset) 

print(dataTest.describe())
dataTest.head()

          latitude    longitude      boro_nm         year        month  \
count  8728.000000  8728.000000  8728.000000  8728.000000  8728.000000   
mean     40.727708   -73.919711     2.429995  2021.173006     7.661664   
std       0.084838     0.085275     1.256277     0.378274     3.706486   
min      40.501464   -74.249755     0.000000  2021.000000     1.000000   
25%      40.666669   -73.968300     1.000000  2021.000000     5.000000   
50%      40.718350   -73.920387     2.000000  2021.000000     9.000000   
75%      40.800838   -73.869419     3.000000  2021.000000    11.000000   
max      40.907060   -73.704599     5.000000  2022.000000    12.000000   

           weekday         Hour       Minute  
count  8728.000000  8728.000000  8728.000000  
mean      2.994500    11.497594    29.511801  
std       2.001195     6.924811    17.327251  
min       0.000000     0.000000     0.000000  
25%       1.000000     5.000000    14.000000  
50%       3.000000    11.000000    30.000000  
75% 

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute
0,40.861921,-73.891913,3,2021.0,1.0,4.0,0.0,0.0
113,40.537644,-74.219184,5,2021.0,1.0,4.0,0.0,3.0
59,40.731657,-74.010306,4,2021.0,1.0,4.0,0.0,33.0
5,40.625462,-73.933187,1,2021.0,1.0,4.0,1.0,3.0
64,40.674682,-73.736538,2,2021.0,1.0,4.0,1.0,37.0


In [10]:
x_train = dataTrain.drop(['law_cat_cd', 'ofns_desc'],axis=1).values
y_train = dataTrain.get(['law_cat_cd', 'ofns_desc']).values

In [11]:
x_train

array([[ 40.74892339, -73.89223005,   2.        , ...,   6.        ,
          8.        ,  50.        ],
       [ 40.65230855, -73.95629004,   1.        , ...,   3.        ,
         22.        ,   0.        ],
       [ 40.84098357, -73.92597392,   3.        , ...,   0.        ,
         23.        ,  47.        ],
       ...,
       [ 40.66844259, -73.80061554,   2.        , ...,   2.        ,
         23.        ,  15.        ],
       [ 40.69346544, -73.92714741,   1.        , ...,   2.        ,
         23.        ,  19.        ],
       [ 40.65448439, -73.96182075,   1.        , ...,   2.        ,
         23.        ,  25.        ]])

In [12]:
y_train = y_train.astype(int)
y_train

array([[ 1,  4],
       [ 1, 11],
       [ 1, 13],
       ...,
       [ 1, 13],
       [ 0, 14],
       [ 2,  7]])

In [13]:
x_test = dataTest.values
x_test

array([[ 40.86192075, -73.89191318,   3.        , ...,   4.        ,
          0.        ,   0.        ],
       [ 40.53764404, -74.21918381,   5.        , ...,   4.        ,
          0.        ,   3.        ],
       [ 40.7316567 , -74.01030637,   4.        , ...,   4.        ,
          0.        ,  33.        ],
       ...,
       [ 40.70998449, -73.99644134,   4.        , ...,   3.        ,
         22.        ,  39.        ],
       [ 40.87328446, -73.84041216,   3.        , ...,   3.        ,
         23.        ,  12.        ],
       [ 40.6647962 , -73.93150058,   1.        , ...,   3.        ,
         23.        ,  43.        ]])

In [14]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(204643, 8)
(204643, 2)
(8728, 8)


In [15]:
# Configure the XGBoost model
param = {'booster': 'gbtree', 
         'tree_method': 'gpu_hist',
         'n_estimators':100,
         'predictor': 'gpu_predictor',
         'max_depth': 50, 
         'eta': 0.1, 
         'objective': '{multi:softmax}', 
         'eval_metric': 'mlogloss',       # merror
         'num_round': 10,
         'feature_selector ': 'cyclic'
        }

In [16]:
def getPredictionResults(param, x_train, y_train, x_test, index):
    
    y_train = y_train.transpose()[index]
    
    global xgb_clf 
    xgb_clf = XGBClassifier(param)
    xgb_clf.fit(x_train, y_train)
#     score = xgb_clf.score(x_test, y_test)
    print("---------------------------------Index = {}-----------------------------------".format(index))

    xgbCpred = xgb_clf.predict(x_test)
    print(xgbCpred)

#     print(y_test)

#     results_log = classification_report(y_test, xgbCpred)
#     print(results_log)
    
    return xgbCpred

In [17]:
law_cat_cd_pred = getPredictionResults(param, x_train, y_train, x_test, 0)

C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\core.py:430: FutureWarning: Pass `objective` as keyword args.  Passing these as positional arguments will be considered as error in future releases.
  warnings.warn(
C:\Users\robinlobo\AppData\Local\Programs\Python\Python39\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[19:52:02] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------Index = 0-----------------------------------
[1 1 1 ... 1 1 0]


In [18]:
ofns_desc_pred = getPredictionResults(param, x_train, y_train, x_test, 1)

[19:52:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
---------------------------------Index = 1-----------------------------------
[11  4 11 ... 11  7  7]


In [19]:
dataXGBpred = pd.DataFrame({'latitude':x_test[:,0] , 'longitude':x_test[:,1] , 'boro_nm':x_test[:,2] , 'year':x_test[:,3] , \
                          'month':x_test[:,4] , 'weekday':x_test[:,5] , 'Hour':x_test[:,6] , 'Minute':x_test[:,7] , \
                          'law_cat_cd_pred':law_cat_cd_pred , 'ofns_desc_pred': ofns_desc_pred})
print(dataXGBpred.describe())
dataXGBpred.head(20)

          latitude    longitude      boro_nm         year        month  \
count  8728.000000  8728.000000  8728.000000  8728.000000  8728.000000   
mean     40.727708   -73.919711     2.429995  2021.173006     7.661664   
std       0.084838     0.085275     1.256277     0.378274     3.706486   
min      40.501464   -74.249755     0.000000  2021.000000     1.000000   
25%      40.666669   -73.968300     1.000000  2021.000000     5.000000   
50%      40.718350   -73.920387     2.000000  2021.000000     9.000000   
75%      40.800838   -73.869419     3.000000  2021.000000    11.000000   
max      40.907060   -73.704599     5.000000  2022.000000    12.000000   

           weekday         Hour       Minute  law_cat_cd_pred  ofns_desc_pred  
count  8728.000000  8728.000000  8728.000000      8728.000000     8728.000000  
mean      2.994500    11.497594    29.511801         0.870990        6.965742  
std       2.001195     6.924811    17.327251         0.344668        3.884855  
min       0.0

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
0,40.861921,-73.891913,3.0,2021.0,1.0,4.0,0.0,0.0,1,11
1,40.537644,-74.219184,5.0,2021.0,1.0,4.0,0.0,3.0,1,4
2,40.731657,-74.010306,4.0,2021.0,1.0,4.0,0.0,33.0,1,11
3,40.625462,-73.933187,1.0,2021.0,1.0,4.0,1.0,3.0,1,11
4,40.674682,-73.736538,2.0,2021.0,1.0,4.0,1.0,37.0,1,11
5,40.679372,-73.912413,1.0,2021.0,1.0,4.0,2.0,7.0,1,6
6,40.646619,-73.996200,1.0,2021.0,1.0,4.0,2.0,41.0,1,11
7,40.512038,-74.249755,5.0,2021.0,1.0,4.0,3.0,11.0,1,11
8,40.760165,-73.819426,2.0,2021.0,1.0,4.0,3.0,45.0,0,7
9,40.709814,-74.009989,4.0,2021.0,1.0,4.0,4.0,15.0,1,11


In [20]:
def convertNumToClass(p_df):
    dict_law_cat_cd1 = dict(zip(dict_law_cat_cd.values(), dict_law_cat_cd.keys()))
    p_df['law_cat_cd_pred']=p_df['law_cat_cd_pred'].astype(int)
    p_df['law_cat_cd_pred']=p_df['law_cat_cd_pred'].map(dict_law_cat_cd1)
    print(dict_law_cat_cd1)

    dict_ofns_desc1 = dict(zip(dict_ofns_desc.values(), dict_ofns_desc.keys()))
    p_df['ofns_desc_pred']=p_df['ofns_desc_pred'].astype(int)
    p_df['ofns_desc_pred']=p_df['ofns_desc_pred'].map(dict_ofns_desc1)
    print(dict_ofns_desc1)

    dict_boro_nm1 = dict(zip(dict_boro_nm.values(), dict_boro_nm.keys()))
    p_df['boro_nm']=p_df['boro_nm'].astype(int)
    p_df['boro_nm']=p_df['boro_nm'].map(dict_boro_nm1)
    print(dict_boro_nm1)

    return p_df

In [21]:
dataXGBpred = convertNumToClass(dataXGBpred)
dataXGBpred.head(20)

{0: 'FELONY', 1: 'MISDEMEANOR', 2: 'VIOLATION'}
{0: 'MURDER & NON-NEGL. MANSLAUGHTER', 1: 'FORGERY', 2: 'GRAND LARCENY', 3: 'FELONY ASSAULT', 4: 'PETIT LARCENY', 5: 'THEFT-FRAUD', 6: 'DANGEROUS WEAPONS', 7: 'HARRASSMENT 2', 8: 'BURGLARY', 9: 'ADMINISTRATIVE CODE', 10: 'SEX CRIMES', 11: 'CRIMINAL MISCHIEF & RELATED OF', 12: 'DANGEROUS DRUGS', 13: 'ASSAULT 3 & RELATED OFFENSES', 14: 'ROBBERY', 15: 'VEHICLE AND TRAFFIC LAWS', 16: 'MISCELLANEOUS PENAL LAW', 17: 'RAPE', 18: 'OFFENSES INVOLVING FRAUD', 19: 'OFF. AGNST PUB ORD SENSBLTY &', 20: 'FRAUDS', 21: 'GRAND LARCENY OF MOTOR VEHICLE', 22: 'OFFENSES AGAINST THE PERSON', 23: 'INTOXICATED & IMPAIRED DRIVING', 24: 'UNAUTHORIZED USE OF A VEHICLE', 25: 'OFFENSES AGAINST PUBLIC ADMINI', 26: 'POSSESSION OF STOLEN PROPERTY', 27: 'CRIMINAL TRESPASS', 28: 'OTHER OFFENSES RELATED TO THEF', 29: 'NYS LAWS-UNCLASSIFIED FELONY', 30: 'OTHER STATE LAWS (NON PENAL LA', 31: "BURGLAR'S TOOLS", 32: 'AGRICULTURE & MRKTS LAW-UNCLASSIFIED', 33: 'ARSON', 34: 'NE

,latitude,longitude,boro_nm,year,month,weekday,Hour,Minute,law_cat_cd_pred,ofns_desc_pred
0,40.861921,-73.891913,BRONX,2021.0,1.0,4.0,0.0,0.0,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF
1,40.537644,-74.219184,STATEN ISLAND,2021.0,1.0,4.0,0.0,3.0,MISDEMEANOR,PETIT LARCENY
2,40.731657,-74.010306,MANHATTAN,2021.0,1.0,4.0,0.0,33.0,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF
3,40.625462,-73.933187,BROOKLYN,2021.0,1.0,4.0,1.0,3.0,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF
4,40.674682,-73.736538,QUEENS,2021.0,1.0,4.0,1.0,37.0,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF
5,40.679372,-73.912413,BROOKLYN,2021.0,1.0,4.0,2.0,7.0,MISDEMEANOR,DANGEROUS WEAPONS
6,40.646619,-73.996200,BROOKLYN,2021.0,1.0,4.0,2.0,41.0,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF
7,40.512038,-74.249755,STATEN ISLAND,2021.0,1.0,4.0,3.0,11.0,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF
8,40.760165,-73.819426,QUEENS,2021.0,1.0,4.0,3.0,45.0,FELONY,HARRASSMENT 2
9,40.709814,-74.009989,MANHATTAN,2021.0,1.0,4.0,4.0,15.0,MISDEMEANOR,CRIMINAL MISCHIEF & RELATED OF


In [22]:
def filter_data(data_pred, user_input = None):
    if user_input == None:
        return data_pred
    else:
        filtered_data = data_pred[data_pred['boro_nm'].str.contains(user_input)]
        return filtered_data
        print(user_input)

In [ ]:
user_input = input("Enter boron name: ")
print(user_input)
m = filter_data(dataXGBpred, user_input)
m

In [ ]:
len(dataXGBpred)

In [ ]:
dataXGBpred.to_csv('finalOutput.csv')